In [5]:
from pandas import read_csv
from torch.optim import SGD
from torch.nn import MSELoss
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from torch import randperm
from config import DATA_DIR
from src.data_set import RatingsDataset
from src.loss import MiningOutliersLoss
from src.model import MF
from src.runner import Runner
from src.utils import (
    create_dataset,
    mine_outliers_scipy,
    mine_outliers_sklearn,
    mine_outliers_torch,
    DataConverter,
    DataProcessor,
    mean_normalized
)


"""
The Deam dataset is based on Arousal-Valence 2D emotional model.
The Valence/Arousal ratings were collected using Amazon Mechanical Turks service.
Each turk from the collected crowd were asked to mark his own emotion for the current song on a 2D plane, Arousal/Valence.
For more information please read: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0173392
"""

DF_PATH = f"{DATA_DIR}" \
          f"/DEAM/annotations/annotations per each rater/" \
          f"song_level/static_annotations_songs_1_2000.csv"

def select_n_random(trainset: RatingsDataset):
    """
    Selects n random data points and their corresponding labels from a dataset
    """
    perm = randperm(len(trainset))
    return trainset[perm][:100]

In [6]:
"""
This block of code calculates the outliers alongside the valence axis
"""
columns = ["workerID", "SongId", "Valence"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
original_df = mean_normalized(dataframe=original_df)

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=9
)

valence_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)
epochs = 50

criterion = MSELoss()
optimizer = SGD(valence_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=valence_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

with SummaryWriter("runs/DEAM/valence") as writer:
    writer.add_graph(valence_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 18/18 [00:00<00:00, 30.62batch/s, train_loss=0.00692]


epoch=1, loss=0.0814835940810317


100%|██████████| 18/18 [00:00<00:00, 46.13batch/s, train_loss=0.00606]


epoch=2, loss=0.06798636331713157


100%|██████████| 18/18 [00:00<00:00, 33.55batch/s, train_loss=0.00481]


...

epoch=48, loss=0.008636535273024321


100%|██████████| 18/18 [00:00<00:00, 25.89batch/s, train_loss=0.000779]


epoch=49, loss=0.00852047487307972


100%|██████████| 18/18 [00:00<00:00, 31.07batch/s, train_loss=0.00088] 

epoch=50, loss=0.0085368808904925


In [7]:
"""
This block of code calculates the outliers alongside the arousal axis
"""
columns = ["workerID", "SongId", "Arousal"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
original_df = mean_normalized(dataframe=original_df)

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=9
)

arousal_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)
epochs = 50

criterion = MSELoss()
optimizer = SGD(arousal_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=arousal_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs,
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

epochs = 50
with SummaryWriter("runs/DEAM/arousal") as writer:
    writer.add_graph(arousal_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 18/18 [00:00<00:00, 30.84batch/s, train_loss=0.00685]


epoch=1, loss=0.07889618623988293


100%|██████████| 18/18 [00:00<00:00, 47.26batch/s, train_loss=0.00564]


epoch=2, loss=0.06499546795717646


100%|██████████| 18/18 [00:00<00:00, 31.14batch/s, train_loss=0.00512]


...

epoch=48, loss=0.008427848756420914


100%|██████████| 18/18 [00:00<00:00, 27.30batch/s, train_loss=0.00085] 


epoch=49, loss=0.00836037932503094


100%|██████████| 18/18 [00:00<00:00, 32.79batch/s, train_loss=0.000882]

epoch=50, loss=0.008336783372430594


In [8]:
valence_outliers = mine_outliers_scipy(model=valence_model, data_converter=data_converter)
arousal_outliers = mine_outliers_scipy(model=arousal_model, data_converter=data_converter)

items_group_by_users = data_converter.original_df.groupby("user_id")
combined_outliers = {}
for user_id, valence_dist in valence_outliers.items():
    arousal_dist = arousal_outliers[user_id]
    combined_outliers[user_id] = valence_dist + arousal_dist

combined_outliers = dict(sorted(combined_outliers.items(), key=lambda item: item[1]))
for user_id, item_id in combined_outliers.items():
    number_of_items = len(items_group_by_users.get_group(user_id))
    print(f"user: {user_id}, dist: {item_id}, #items: {number_of_items}")

user: ff18a27328ffd40ef52b7ebb7a0ded94, dist: -74.27330861100704, #items: 20
user: 19fee46f2810f34a8b69a7768d897a59, dist: -53.744675938102034, #items: 1
user: random_guy_191, dist: -47.90946700519498, #items: 9
user: random_guy_190, dist: -46.617390778418574, #items: 9
user: random_guy_188, dist: -43.83338179209181, #items: 9
user: 3111e02887b600ee085c72c0a3df33e8, dist: -36.50610478373544, #items: 1
user: random_guy_189, dist: -35.82082827645257, #items: 9
user: fd5b08ce362d855ca9152a894348130c, dist: -35.6837828862546, #items: 222
user: 2f790705ae66e70e81cc0f11ce0f4b9b, dist: -32.06156850854283, #items: 2
user: b8ef6a913a63225faafd661ee2e1a7c0, dist: -23.481714259168395, #items: 10
user: e105c200f413d7b2c5850c0df4b9687e, dist: -23.32396391727844, #items: 2
user: 7cecbffe1da5ae974952db6c13695afe, dist: -19.721673771900445, #items: 428
user: 027cefa6afc040448d29558b3175cdc1, dist: -9.499037434780577, #items: 9
user: 8eb1abd1acca601d1e23e85c69b1742a, dist: -6.8273288473123115, #items: 